# TP 7 : Random Forest

# <div class="alert alert-block alert-success">
<b>Informations générales </b>
</div>

**Professeur**: Stephan Robert

**Assistant(s)**: Félicien Hêche

**Contact**: stephan.o.robert@gmail.com, felicien.heche@gmail.com ou via Teams

**Rendu**:

- Date: 9 Janvier 2024
- Modalité: Travail individuel. Une fois complétées, copiez les réponses dans le ficher "**TP7_APV_S1-2023_Nom_Prenom.*****" (*** = ce que vous voulez qui soit lisible pour nous: .pdf, .html, ...) en remplaçant Nom et Prenom par les votres puis uploader votre fichier sur Cyberlearn.
- Note: Ce TP est noté sur 6, pour un poids de x%

**Étudiant**:

- Prénom Nom

<div class="alert alert-block alert-success">

<b>But du TP </b>
</div>

Le but de ce travail pratique est d'implémenter un algorithme appelé Random Forest! Dans ce travail pratique, nous allons travailler avec un dataset trouvé sur kaggle. Ce dataset est assez brute et par conséquent, il faudra faire un peu de préprocessing avant de pouvoir l'utiliser.


<!---
<div class="alert alert-block alert-info">
<b>Rappel </b>
</div>
-->

<div class="alert alert-block alert-success">
<b>Table des matières </b>
</div>

1. Data préparation
2. Random Forest
3. Test

Commencons par importer quelques packages.

In [1]:
import sklearn
import numpy as np
from tqdm import tqdm
from statistics import mode
from sklearn.tree import DecisionTreeRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import pandas as pd
import re 

## 1. Data préparation

Pour ce TP, nous allons travailler avec un dataset qui se nomme 'Australian Housing Prices'. Ce dataset vient de kaggle (plus d'information [ici](https://www.kaggle.com/datasets/thedevastator/australian-housing-data-1000-properties-sampled)). Ce fichier contient 27 features concernant $1000$ maisons/appartements australiens ainsi que le prix de vente de la maison/appartement en question. Dans ce travail pratique, nous allons utiliser l'algorithme de random forest afin d'estimer le prix d'une maison/appartement.

Commençons par essayer de comprendre un peu mieux le dataset.

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Comme vous pouvez le voir, notre dataset n'est pas utilisable comme tel, car il a des données manquantes et des features de type string. Nous allons donc commencer par préparer notre dataset.

### 1.1 Gestion des données manquantes

Pour commencer nous allons gérer nos données manquantes. En effet, comme nous pouvons le constater, il y a actuellememt un grand nombre de données manquantes dans notre jeux de données.


In [3]:
# TODO : count the number of houses with missing feature.

Acutally, the are 1460 missing data.


En première approche, nous allons donc déterminer et supprimer les features de notre dataset qui ont un nombre de valeurs manquantes trop élevées.

In [ ]:
# TODO : find the feature with more than 200 missing values and delete them from the dataset.

Ensuite, il est également, préférable de supprimer la colonne 'Id'.

In [6]:
# TODO delete the row 'Id'

In [7]:
# TODO : count the number houses with missing information in our dataset.   

Acutally, there are 122 missing data.


Bien que le nombre de données manquantes ait significativement diminué, il reste encore quelques données manquantes. Pour finir, nous allons donc supprimer les lignes dans lesquelles il y a encore des données manquantes.

In [8]:
# TODO : delete houses sample which still contains missing data

In [9]:
# # TODO : check there is no missing data anymore.  

Acutally, the are 0 missing data.


## 1.3 Numérisation

Maintenant que nous avons géré le problème des données manquantes, passons au problème suivant. Comme vous avez pu le constater toutes les valeurs présentes dans notre dataset ne sont pas de type numérique. Cela est évidemment problématique, car nous devons nourrir notre modèle avec des données de ce type.

Nous allons donc gérer ce problème ici.

In [10]:
# TODO : transform string feature into numeric value

Maintenant, nous pouvons convertir nos données string en données numéric.

In [11]:
df = numerise_data(df)

Evidemment, nous pouvons vérifier que tout a bien fonctionné.

In [12]:
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,0,8450,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,20,0,9600,0,0,0,0,1,0,1,...,0,0,0,0,0,5,2007,0,0,181500
2,60,0,11250,0,1,0,0,0,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,70,0,9550,0,1,0,0,2,0,2,...,272,0,0,0,0,2,2006,0,1,140000
4,60,0,14260,0,1,0,0,1,0,3,...,0,0,0,0,0,12,2008,0,0,250000


Evidemment, la manière dont nous avons préparé le dataset est très simpliste. Si nous étions intéressé à construire un modèle performant, il aurait pu être pertinent d'approfondir cette étape (détection de potentiel valeurs aberrantes, gestion de ces derniers, feature sélecion, ...).

### 1.4 Data splitting

Maintenant que notre dataset est prêt, nous pouvons construire notre training et test dataset.

In [13]:
# TODO : split the dataset into the training, evaluation and test dataset.
# We can use 70% of the data for the training, 15% for the evaluation and 15% for the test.


## 2. Random Forest

Pour continuer, nous pouvons travailler sur l'implémentation de l'algorithme de Random Forest. L'idée de cet algorihme est très simple : construire un ensemble d'arbre de décision où chaque arbre sera entraînés sur dataset différents. Pour créer ces différents datasets, l'approche classique consiste à faire du bootstraping. 

De plus, la manière de construire ces arbres est un peu spécifique. En effet lors de la construction, pour choisir le prochain noeud, on va considérer le meilleur feature parmi un certain sous-ensemble aléatoirement choisi de features. Plus précisément s'il y a $n$ features possible, il est courant de choisir aléatoirerement $\sqrt n$ features et de choisir le meilleur parmi ces $\sqrt n$ features.

Finalement, une fois que notre Random Forest a été construit et que nous souhaitons faire une régression, il suffit de considérer la moyenne des différentes prédictions. Dans le cas d'une classification, il suffit de choisir la classe avec le plus de votes.

In [14]:
class RandomForest:
    """
    An implementation of Random Forest algorithm
    """
        """
    An implementation of Random Forest algorithm
    """
    # TODO : implement this class -> Of course you can use DecisionTreeRegressor provided by sklearn
    
    

## 3. Test

Pour finir, nous allons construire et tester notre algorithme.

### 3.1 Hyperparameters tuning

En utilisant le training et le validation dataset précédemment, faites un peu d'hypermeter tuning. En particulier, tester quelques valeurs différentes pour le nombre d'arbre présents. Evaluer la performance de votre modèle en utilisant l'erreur absolue moyenne.

In [ ]:
# TODO : tune some hyperparmeters

### 3.2 Evaluation du modèle et comparaisons

En utilisant les meilleurs paramètres trouvé au point précédent, construiser un modèle et évaluer son erreur absolue moyenne sur le test dataset.

In [ ]:
# TODO : build and test the best model using parameters found above on the test dataset.

Ensuite, comparer votre modèle avec un arbre de décision.

In [ ]:
# TODO : build and test a simple decision tree.

Pour finir, comparer la performance de votre modèle avec un RandomForestRegressor fourni par sklearn. Comme des processus aléatoires sont utilisés pour construire ces modèles, il est normal d'obtenir des résultats qui diffèrent un peu.

In [ ]:
# Build and test a RandomForestRegressor model provided by sklearn